# カテゴリ値を含むデータでのLightGBMを使った分析

### データ
[Adult Data Set](https://archive.ics.uci.edu/ml/datasets/adult) \
[Adult Census Income](https://www.kaggle.com/datasets/uciml/adult-census-income) \
[Adult income dataset](https://www.kaggle.com/datasets/wenruliu/adult-income-dataset)

### 参考
[lightgbmで二値分類の一連の流れをしたメモ](https://qiita.com/d_desuyon/items/807e01311ad08570ee78) \
[【初心者向け】特徴量重要度の算出 (LightGBM) 【Python】【機械学習】](https://mathmatical22.xyz/2020/04/12/%E3%80%90%E5%88%9D%E5%BF%83%E8%80%85%E5%90%91%E3%81%91%E3%80%91%E7%89%B9%E5%BE%B4%E9%87%8F%E9%87%8D%E8%A6%81%E5%BA%A6%E3%81%AE%E7%AE%97%E5%87%BA-lightgbm-%E3%80%90python%E3%80%91%E3%80%90%E6%A9%9F/) \
[LightGBMのCategorical Featureによって精度が向上するか？](https://qiita.com/sinchir0/items/b038757e578b790ec96a) \
[Pythonでしか描けない美しいデシジョンツリー（決定木）をlightGBMでも描こう！](https://qiita.com/hima2b4/items/b356e688ebdabc393da8) \
[LightGBM/XGBoostのtree構造可視化メモ](https://qiita.com/dyamaguc/items/70d4ce3548d49a6256fa) \
[Category Encodersでカテゴリ特徴量をストレスなく変換する](https://qiita.com/Hyperion13fleet/items/afa49a84bd5db65ffc31) \
[カテゴリ変数系特徴量の前処理(scikit-learnとcategory_encoders)](https://qiita.com/FukuharaYohei/items/06cc7b1c5eb124db6cdf) \
[LightGBMのCategorical Featureによって精度が向上するか？](https://qiita.com/sinchir0/items/b038757e578b790ec96a) \
[LightGBMでOptunaを使用するときの再現性確保について](https://book-read-yoshi.hatenablog.com/entry/2021/03/22/lightgbm_optuna_deterministic)

In [1]:
!python -V

Python 3.9.13


In [2]:
!python -m pip --version

pip 23.0.1 from C:\Python39\lib\site-packages\pip (python 3.9)



In [3]:
!python -m pip list

Package                  Version
------------------------ -----------
aiofiles                 22.1.0
aiosqlite                0.18.0
alembic                  1.9.4
anyio                    3.6.2
argon2-cffi              21.3.0
argon2-cffi-bindings     21.2.0
arrow                    1.2.3
asttokens                2.2.1
attrs                    22.2.0
Babel                    2.11.0
backcall                 0.2.0
beautifulsoup4           4.11.2
bleach                   6.0.0
bs4                      0.0.1
category-encoders        2.6.0
certifi                  2022.12.7
cffi                     1.15.1
charset-normalizer       3.0.1
cmaes                    0.9.1
colorama                 0.4.6
colorlog                 6.7.0
comm                     0.1.2
contourpy                1.0.7
cycler                   0.11.0
debugpy                  1.6.6
decorator                5.1.1
defusedxml               0.7.1
executing                1.2.0
fastjsonschema           2.16.2
fonttools        

In [4]:
import pandas as pd
pd.set_option('display.max_rows', 100)
import numpy as np
from sklearn.model_selection import train_test_split
#import lightgbm as lgb
import optuna
# ログを非表示にしたいときは、optuna.logging.disable_default_handler()
# 逆に、表示にしたいときは、optuna.logging.enable_default_handler()
optuna.logging.disable_default_handler()
# optuna経由でLightGBMをインポート
from optuna.integration import lightgbm as lgb
from sklearn.metrics import accuracy_score, f1_score, auc, roc_curve, precision_score, confusion_matrix, ConfusionMatrixDisplay
from matplotlib import pyplot as plt
%matplotlib inline

In [5]:
df = pd.read_csv('adult_uci_onehot.csv')

In [6]:
df.head(10)

,age,workclass_1,workclass_2,workclass_3,workclass_4,workclass_5,workclass_6,workclass_7,fnlwgt,education_1,...,native.country_33,native.country_34,native.country_35,native.country_36,native.country_37,native.country_38,native.country_39,native.country_40,native.country_41,income
0,82,1,0,0,0,0,0,0,132870,1,...,0,0,0,0,0,0,0,0,0,0
1,54,1,0,0,0,0,0,0,140359,0,...,0,0,0,0,0,0,0,0,0,0
2,41,1,0,0,0,0,0,0,264663,0,...,0,0,0,0,0,0,0,0,0,0
3,34,1,0,0,0,0,0,0,216864,1,...,0,0,0,0,0,0,0,0,0,0
4,38,1,0,0,0,0,0,0,150601,0,...,0,0,0,0,0,0,0,0,0,0
5,74,0,1,0,0,0,0,0,88638,0,...,0,0,0,0,0,0,0,0,0,1
6,68,0,0,1,0,0,0,0,422013,1,...,0,0,0,0,0,0,0,0,0,0
7,45,1,0,0,0,0,0,0,172274,0,...,0,0,0,0,0,0,0,0,0,1
8,38,0,0,0,1,0,0,0,164526,0,...,0,0,0,0,0,0,0,0,0,1
9,52,1,0,0,0,0,0,0,129177,0,...,0,0,0,0,0,0,0,0,0,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30162 entries, 0 to 30161
Columns: 105 entries, age to income
dtypes: int64(105)
memory usage: 24.2 MB


In [8]:
df['income'].value_counts()

0    22654
1     7508
Name: income, dtype: int64

In [9]:
# 不要カラムの削除と並べ替え
'''
df = df[["age",
         "workclass",
         #"fnlwgt",
         "education",
         "education.num",
         "marital.status",
         "occupation",
         "relationship",
         "race",
         "sex",
         "capital.gain",
         "capital.loss",
         "hours.per.week",
         "native.country",
         "income",]]
'''
df = df.drop(["fnlwgt"], axis=1)
df.head(10)

,age,workclass_1,workclass_2,workclass_3,workclass_4,workclass_5,workclass_6,workclass_7,education_1,education_2,...,native.country_33,native.country_34,native.country_35,native.country_36,native.country_37,native.country_38,native.country_39,native.country_40,native.country_41,income
0,82,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,54,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,41,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,34,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,38,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,74,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,68,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,45,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,38,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,52,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [10]:
X = df.drop(['income'], axis=1)
y = df['income']

#Creation of Train and Test dataset
X_train, X_test , y_train, y_test  = train_test_split(X      , y      , test_size=0.2, shuffle=False)

#Creation of Train and validation dataset
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)

In [11]:
print ("Train dataset: {0}{1}".format(X_train.shape, y_train.shape))
print ("Validation dataset: {0}{1}".format(X_valid.shape, y_valid.shape))
print ("Test dataset: {0}{1}".format(X_test.shape, y_test.shape))

Train dataset: (19303, 103)(19303,)
Validation dataset: (4826, 103)(4826,)
Test dataset: (6033, 103)(6033,)


In [12]:
# データセットを生成する
lgb_train = lgb.Dataset(X_train, y_train)
lgb_valid = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

In [13]:
# LightGBMのハイパーパラメータ
params = {
    # 勾配ブースティング木(GBDT:Gradient Boosting Decision Tree)
    'boosting_type': 'gbdt',
    # 二値分類問題
    'objective': 'binary',
    # AUCの最大化を目指す
    'metric': 'auc',
    # 正答率の最大化を目指す
    #'metric': 'binary_error',
    # Fatal の場合出力
    'verbosity': -1,
    # 学習率
    'learning_rate': 0.02,
    'deterministic': True, # 再現性確保用のパラメータ
    'force_row_wise': True  # 再現性確保用のパラメータ
}

In [14]:
# 乱数Seedを指定した試行
# LightGBMTunerのインスタンス化、学習（≒探索）、最適モデル（およびパラメータ）取得などの流れは前節に同じ
# 更にモデル精度指標算出を追加
def autotuning(seed=31):
    booster = lgb.LightGBMTuner(
        params = params,
        train_set = lgb_train,
        valid_sets = lgb_valid,
        verbose_eval = 50,  # 50イテレーション毎に学習結果出力
        num_boost_round = 10000,  # 最大イテレーション回数指定
        early_stopping_rounds = 100,
        optuna_seed = seed, # 再現性確保用のパラメータ
    )

    # 上記のパラメータでモデルを学習する
    booster.run()

    # 最適なパラメータの表示
    #print("Best params:", booster.best_params)

    # 最適なモデル（Boosterオブジェクト）を取得する
    model = booster.get_best_booster()

    # 最適なパラメータの表示
    best_params = model.params
    print("Best params:", best_params)

    # テストデータを予測する
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    # AUCを計算
    fpr, tpr, thresholds = roc_curve(np.asarray(y_test), y_pred)
    #print("AUC", auc(fpr, tpr))

    metrics = {
        "seed": seed,
        "auc": auc(fpr, tpr),
        "accuracy": accuracy_score(np.asarray(y_test), np.round(y_pred)),
        "precision": precision_score(np.asarray(y_test), np.round(y_pred)),
    }

    # Tupleとして戻り値に
    return (best_params, metrics)

In [15]:
# 乱数シードを指定（from 素数を取得するワンライナー）
seeds = (lambda n:[x for x in range(2, n) if not 0 in map(lambda z: x%z, range(2, x))])(10)

In [16]:
import csv

# 各試行での情報を保存
listParams = []
listMetrics = []

# 乱数Seedを指定した試行を自動化
for s in seeds:
    print("### seed:%3s ###" %s)
    tunedparam, metrics = autotuning(s)
    tunedparam["seed"] = s
    listParams.append(tunedparam)
    listMetrics.append(metrics)

# DataFrameからCSVファイルを生成
# encoding="SJIS"だとJupyterLab（CSVTable）上で表示不可なことに注意
df_params = pd.DataFrame(listParams)
df_params.to_csv('lightgbm_onehot_params.csv', index=False, header=True, encoding='UTF-8', sep=',', quotechar='"', quoting=csv.QUOTE_ALL)
df_metrics = pd.DataFrame(listMetrics)
df_metrics.to_csv('lightgbm_onehot_metrics.csv', index=False, header=True, encoding='UTF-8', sep=',', quotechar='"', quoting=csv.QUOTE_ALL)

### seed:  2 ###


feature_fraction, val_score: -inf:   0%|                                                         | 0/7 [00:00<?, ?it/s]C:\Python39\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Python39\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


Training until validation scores don't improve for 100 rounds
[50]	valid_0's auc: 0.895481
[100]	valid_0's auc: 0.899061
[150]	valid_0's auc: 0.901728
[200]	valid_0's auc: 0.90346
[250]	valid_0's auc: 0.904153
[300]	valid_0's auc: 0.904424
[350]	valid_0's auc: 0.904549
[400]	valid_0's auc: 0.904709
[450]	valid_0's auc: 0.904767
[500]	valid_0's auc: 0.904665


feature_fraction, val_score: 0.904811:  14%|######4                                      | 1/7 [00:01<00:06,  1.16s/it]

Early stopping, best iteration is:
[434]	valid_0's auc: 0.904811
Training until validation scores don't improve for 100 rounds


...

[600]	valid_0's auc: 0.906544
Early stopping, best iteration is:
[505]	valid_0's auc: 0.906605
Training until validation scores don't improve for 100 rounds


C:\Python39\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Python39\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[50]	valid_0's auc: 0.896579
[100]	valid_0's auc: 0.900298
[150]	valid_0's auc: 0.902498
[200]	valid_0's auc: 0.903953
[250]	valid_0's auc: 0.905009
[300]	valid_0's auc: 0.905552
[350]	valid_0's auc: 0.905933
[400]	valid_0's auc: 0.906071
[450]	valid_0's auc: 0.906277
[500]	valid_0's auc: 0.906386
[550]	valid_0's auc: 0.906442
[600]	valid_0's auc: 0.906511


min_data_in_leaf, val_score: 0.907403: 100%|#############################################| 5/5 [00:06<00:00,  1.37s/it]

[650]	valid_0's auc: 0.906475
[700]	valid_0's auc: 0.906321
Early stopping, best iteration is:
[617]	valid_0's auc: 0.906564
Best params: {'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 'verbosity': -1, 'learning_rate': 0.02, 'deterministic': True, 'force_row_wise': True, 'feature_pre_filter': False, 'lambda_l1': 8.326609203905651e-07, 'lambda_l2': 0.00011725075728346999, 'num_leaves': 20, 'feature_fraction': 0.4, 'bagging_fraction': 0.8075379976732435, 'bagging_freq': 6, 'min_child_samples': 5, 'num_iterations': 10000, 'early_stopping_round': 100}


In [17]:
# 乱数シードで照準にソート
df_params.sort_values("seed", ascending=True)

,boosting_type,objective,metric,verbosity,learning_rate,deterministic,force_row_wise,feature_pre_filter,lambda_l1,lambda_l2,num_leaves,feature_fraction,bagging_fraction,bagging_freq,min_child_samples,num_iterations,early_stopping_round,seed
0,gbdt,binary,auc,-1,0.02,True,True,False,1.259099e-08,0.046383,8,0.48,0.579793,2,10,10000,100,2
1,gbdt,binary,auc,-1,0.02,True,True,False,0.000000e+00,0.000000,9,0.40,0.935768,7,5,10000,100,3
2,gbdt,binary,auc,-1,0.02,True,True,False,2.487143e-04,0.003204,12,0.40,0.859545,4,10,10000,100,5
3,gbdt,binary,auc,-1,0.02,True,True,False,8.326609e-07,0.000117,20,0.40,0.807538,6,5,10000,100,7


In [18]:
# AUCで降順にソート
df_metrics.sort_values("auc", ascending=False)

,seed,auc,accuracy,precision
3,7,0.894543,0.855627,0.681619
0,2,0.894479,0.853473,0.678611
2,5,0.894344,0.855793,0.683168
1,3,0.894129,0.855462,0.685330
